In [1]:
from keras import layers
from keras import models
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint
import os
from jqdatasdk import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
#数据获取及数据处理
auth("xxxxxxx", "xxxxxxxxxx")

#获取A股所有股票的代码
stock_list = list(get_all_securities(["stock"]).index)
len(stock_list)

auth success 


4723

In [4]:
x_data = {}
y_data = {}
attris = ['open', 'close', 'high', 'low', 'volume', 'money']
for attri in attris:
    x_data[attri] = []
    y_data[attri] = []
    
for code in stock_list[0:3000]:
    price_previous = get_price(code, end_date = '2021-11-30', frequency = 'daily',  count = 30)
    price_next = get_price(code, end_date = '2021-12-10', frequency = 'daily',  count = 10)
    
    for attri in attris:
        x_data[attri].append(np.array(price_previous[attri]))
        y_data[attri].append(np.array(price_next[attri]))


print(x_data['open'][1:10])

[array([20.36, 20.2 , 21.06, 20.8 , 20.59, 19.61, 19.6 , 18.96, 18.17,
       18.14, 17.7 , 18.01, 18.  , 18.38, 18.26, 18.37, 18.95, 20.5 ,
       19.56, 19.29, 19.23, 19.35, 19.01, 19.9 , 19.19, 19.12, 19.22,
       19.15, 18.75, 18.44]), array([19.39, 19.  , 18.6 , 18.9 , 18.75, 17.34, 16.3 , 16.53, 16.78,
       16.88, 16.34, 16.67, 16.8 , 17.11, 17.23, 17.4 , 17.42, 17.27,
       17.52, 19.5 , 20.  , 19.1 , 18.84, 18.58, 18.88, 18.8 , 19.1 ,
       18.59, 18.58, 18.56]), array([2.22, 2.26, 2.3 , 2.26, 2.25, 2.22, 2.17, 2.16, 2.18, 2.15, 2.06,
       2.09, 2.08, 2.09, 2.12, 2.14, 2.12, 2.13, 2.12, 2.18, 2.15, 2.17,
       2.15, 2.14, 2.12, 2.13, 2.13, 2.14, 2.14, 2.15]), array([4.44, 4.44, 4.45, 4.39, 4.25, 4.22, 4.13, 4.08, 4.18, 4.16, 4.08,
       4.13, 4.1 , 4.09, 4.09, 4.09, 4.16, 4.23, 4.2 , 4.21, 4.16, 4.17,
       4.12, 4.18, 4.13, 4.16, 4.2 , 4.16, 4.09, 4.09]), array([4.24, 4.09, 4.08, 4.38, 4.25, 4.13, 4.22, 4.21, 4.4 , 4.64, 4.58,
       4.97, 5.05, 4.93, 5.4 , 5.5 , 5.4

In [ ]:
def NAN_Replace(x_data, y_data):
    for i in range(x_data.shape[0]):
        temp_row = x_data[i, :]
        nan_num = np.count_nonzero(temp_row != temp_row)
        if nan_num != 0:
            # temp_row[np.isnan(temp_row)] = data[~np.isnan(data)].mean()
            temp_not_nan_row = temp_row[temp_row == temp_row]  # 去掉nan的ndarray
            # average = 0
            if np.isnan(temp_not_nan_row.mean()):
                average = 0
            else:
                average = temp_not_nan_row.mean()
             
            # 选中当前为nan的位置，把值赋值为不为nan的均值
            temp_row[np.isnan(temp_row)] = average  # mean()表示求均值。
        for j in range(y_data.shape[1]):    
            if np.isnan(y_data[i][j]):
                y_data[i][j] = x_data[i, -1].mean()
            
    
    return x_data, y_data

In [ ]:
# print(x_data[1:10])
x_data_array, y_data_array = NAN_Replace(np.array(x_data['open']), np.array(y_data['open']))
len(y_data)

In [ ]:
x_train = x_data_array[:2500]
print(x_train.shape)
x_train = x_train.reshape((2500, 30, 1))
y_train = y_data_array[:2500]
# y_train = y_train[:, :5]
y_train = y_train.reshape((2500, 10, 1))
x_val = x_data_array[2500:]
x_val = x_val.reshape((500, 30, 1))
y_val = y_data_array[2500:]
# y_val = y_val[:, :5]
y_val = y_val.reshape((500, 10, 1))


In [ ]:
def model(shape, x_train, y_train, x_val, y_val, epochs):
    # LSTM 神经网络搭建
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(20, input_shape=(30, 1), return_sequences=True))
    model.add(layers.Dropout(0.2))
    model.add(tf.keras.layers.LSTM(20))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(10, activation='relu'))
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss="mean_squared_error")
    model.fit(x_train, y_train, epochs=epochs, batch_size= 128, validation_data=(x_val,y_val))
    
    return model

In [ ]:
model = model([30, 1], x_train, y_train, x_val, y_val, 25)

In [ ]:
#使用训练好的模型预测时间序列
code = stock_list[4135]
x = get_price(code, end_date = '2021-11-10', frequency = 'daily',  count = 30)
x_test = np.array(x['open']).reshape((1, 30, 1))
# print(x_test)
y = get_price(code, end_date = '2021-11-20', frequency = 'daily',  count = 10)
y_predict = model.predict(x_test)
y = np.array(y['open'])
print(y_predict[0])
print(y)

In [ ]:
import matplotlib.pyplot as plt
from pylab import *                                 #支持中文
mpl.rcParams['font.sans-serif'] = ['SimHei']
x = [i for i in range(10)]
plt.figure(figsize = (16, 9))
plt.ylim(0, 50)
plt.plot(x, y)
plt.plot(x, y_predict[0])
# plt.legend()
plt.show()

In [ ]:
price_next